In [1]:
from torch.optim.lr_scheduler import ReduceLROnPlateau, CyclicLR
from load_utils import prepare_data
from train_utils import *
from seq2seq_model import *
import numpy as np
import torch
from tqdm.notebook import tnrange, tqdm_notebook
from Calculate_BLEU import *
import datetime

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"

In [3]:
q, a, pairs, vector = prepare_data('train', 'glove.42B.300d/glove.42B.300d.txt', small=True)

Reading train -------
Read 45544 sentence pairs
Counting words
Counted words:
In questions: 16871 words
In answers: 14636 words


In [4]:
# Add EOS at end of each target sentence
pairs = [[line[0], line[1]+' EOS'] for line in pairs ]

In [5]:
matrix_len = q.n_words
weights_matrix = np.zeros((matrix_len, 300))
word_found = 0
for i, word in enumerate(q.word2index):
    try:
        weights_matrix[i] = vector[word]
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(300, ))

In [6]:
criterion = nn.NLLLoss()
optimizer = 'Adam'
batch_size = 32
lr = 0.0005
optimal_so_far = 'Adam and 0.0005'

In [7]:
model = Seq2Seq(batch_size, q.n_words, a.n_words, 300, 100, weights_matrix, 0.2, device, criterion)

C:\Users\maren\Anaconda3\envs\deeplearning\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [8]:
if optimizer == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
else:
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [9]:
#scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, threshold=0.001, min_lr=1e-6)
#scheduler = CyclicLR(optimizer, base_lr=0.001, max_lr=0.1, step_size_up=500)

In [10]:
a.n_words

14636

In [11]:
from jupyterplot import ProgressPlot

pp = ProgressPlot(x_label='Epoch', plot_names=['train_loss', 'train_accuracy'])

for epoch in tnrange(1000, desc="Total epochs: "):

    print(f'Epoch {epoch + 1}: {datetime.datetime.now()}')

    # Calculte loss
    loss = epoch_train(model, optimizer, batch_size, pairs, q, a, device)
    
    #current_lr = optimizer.param_groups[0]['lr']
    
    print(f'Loss: {loss}')
    #print(f'LR: {current_lr}')
    
    #scheduler.step(loss)
    #scheduler.step()

    # Calculate accuracy
    accuracy = epoch_accuray(model, batch_size, pairs, q, a, device)

    print(f'Train accuracy: {accuracy}')

    # Calculate BLEU Score
    #BLEU_model = CalculateBleu(model, batch_size, pairs, q, a, device)
    #bleu_score = BLEU_model.score()

    #print(f'BLUE score: {bleu_score}')

    # Try to do interactive plot
    pp.update([[loss], [accuracy]])

pp.finalize()
print(f"Optimization ended successfully")


Total epochs:   0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1: 2021-09-19 19:08:40.485112
Loss: 2.140199106511594
Train accuracy: 0.10797148971289404
Epoch 2: 2021-09-19 19:11:27.599489
Loss: 1.733134041235318
Train accuracy: 0.10802482368642675
Epoch 3: 2021-09-19 19:14:15.088350
Loss: 1.6415616729534666
Train accuracy: 0.11785270990230265
Epoch 4: 2021-09-19 19:17:01.072178
Loss: 1.6003499773287853
Train accuracy: 0.11908668025765601
Epoch 5: 2021-09-19 19:19:48.148329
Loss: 1.562569699281861
Train accuracy: 0.1506765508486045
Epoch 6: 2021-09-19 19:22:36.662328
Loss: 1.5511648207122053
Train accuracy: 0.1580906393225525
Epoch 7: 2021-09-19 19:25:22.182040
Loss: 1.5332732203216806
Train accuracy: 0.14240700788046914
Epoch 8: 2021-09-19 19:28:08.101147
Loss: 1.534407223273687
Train accuracy: 0.1432473818465309
Epoch 9: 2021-09-19 19:30:54.023717
Loss: 1.5024484570499532
Train accuracy: 0.16377689175322377
Epoch 10: 2021-09-19 19:33:39.492430
Loss: 1.5048923485271608
Train accuracy: 0.16724457058669442
Epoch 11: 2021-09-19 19:36:25.440944

Loss: 1.1637751181034404
Train accuracy: 0.20085757730801448
Epoch 86: 2021-09-19 23:03:52.001745
Loss: 1.1945359106831004
Train accuracy: 0.2009604454068878
Epoch 87: 2021-09-19 23:06:37.913036
Loss: 1.1634843407441269
Train accuracy: 0.2040508451662615
Epoch 88: 2021-09-19 23:09:24.328174
Loss: 1.1752813215516993
Train accuracy: 0.20301940148682576
Epoch 89: 2021-09-19 23:12:10.399687
Loss: 1.150371384334062
Train accuracy: 0.20357334184939269
Epoch 90: 2021-09-19 23:14:57.150904
Loss: 1.1676449256718247
Train accuracy: 0.2042928852972279
Epoch 91: 2021-09-19 23:17:43.219325
Loss: 1.192084284115274
Train accuracy: 0.20247366156150745
Epoch 92: 2021-09-19 23:20:29.350303
Loss: 1.1573299836908313
Train accuracy: 0.1999983844226313
Epoch 93: 2021-09-19 23:23:15.895329
Loss: 1.1514142621847876
Train accuracy: 0.2072379633316575
Epoch 94: 2021-09-19 23:26:02.290992
Loss: 1.1748110421397089
Train accuracy: 0.21439483811956409
Epoch 95: 2021-09-19 23:28:49.114184
Loss: 1.1627602773791166
Tr

Epoch 169: 2021-09-20 02:54:21.284276
Loss: 1.0741564523593694
Train accuracy: 0.2247768174928899
Epoch 170: 2021-09-20 02:57:06.558449
Loss: 1.0643911261438648
Train accuracy: 0.23045963126968994
Epoch 171: 2021-09-20 02:59:51.153460
Loss: 1.0791129508415704
Train accuracy: 0.23364143849390978
Epoch 172: 2021-09-20 03:02:36.318651
Loss: 1.0832747719769398
Train accuracy: 0.2329903120861704
Epoch 173: 2021-09-20 03:05:24.419761
Loss: 1.0871596155403773
Train accuracy: 0.23338631407204646
Epoch 174: 2021-09-20 03:08:09.376156
Loss: 1.076853385388666
Train accuracy: 0.2318610395095221
Epoch 175: 2021-09-20 03:10:54.560325
Loss: 1.0823326390116637
Train accuracy: 0.23005970704089804
Epoch 176: 2021-09-20 03:13:39.332103
Loss: 1.0701952055419672
Train accuracy: 0.23234750498479745
Epoch 177: 2021-09-20 03:16:24.351896
Loss: 1.0727492938996879
Train accuracy: 0.23158324149722137
Epoch 178: 2021-09-20 03:19:09.375361
Loss: 1.0417787640630198
Train accuracy: 0.23208224702313665
Epoch 179: 202

Loss: 1.0397204782785188
Train accuracy: 0.24442381783346231
Epoch 253: 2021-09-20 06:45:12.073316
Loss: 1.0302333918563242
Train accuracy: 0.24757554733869105
Epoch 254: 2021-09-20 06:47:56.883057
Loss: 1.0336443662851524
Train accuracy: 0.24676264130229944
Epoch 255: 2021-09-20 06:50:41.687056
Loss: 1.0440787126230746
Train accuracy: 0.24568391864213923
Epoch 256: 2021-09-20 06:53:27.116733
Loss: 1.0446298995899654
Train accuracy: 0.25379580911598276
Epoch 257: 2021-09-20 06:56:12.087627
Loss: 1.054269856192385
Train accuracy: 0.24808737448374196
Epoch 258: 2021-09-20 06:58:56.751768
Loss: 1.043666896404143
Train accuracy: 0.2487370374610065
Epoch 259: 2021-09-20 07:01:41.339995
Loss: 1.0472348713108988
Train accuracy: 0.2449837176000425
Epoch 260: 2021-09-20 07:04:26.266149
Loss: 1.043203586710666
Train accuracy: 0.24832264969497372
Epoch 261: 2021-09-20 07:07:12.114089
Loss: 1.0405025471021776
Train accuracy: 0.2500360711324864
Epoch 262: 2021-09-20 07:09:57.023061
Loss: 1.04113053

...

Epoch 587: 2021-09-20 22:01:32.300317
Loss: 0.9550788771912251
Train accuracy: 0.276923888689003
Epoch 588: 2021-09-20 22:04:19.800398
Loss: 0.9742679211546948
Train accuracy: 0.2779215971020477
Epoch 589: 2021-09-20 22:07:06.839186
Loss: 0.988675930828417
Train accuracy: 0.2734104249064362
Epoch 590: 2021-09-20 22:09:54.100500
Loss: 0.9610896783660541
Train accuracy: 0.28051368182110287
Epoch 591: 2021-09-20 22:12:41.094170
Loss: 0.9679959594591729
Train accuracy: 0.27827083642220274
Epoch 592: 2021-09-20 22:15:28.529955
Loss: 0.9722122309090281
Train accuracy: 0.2795544470483615
Epoch 593: 2021-09-20 22:18:14.971615
Loss: 0.9951426276349336
Train accuracy: 0.27796066856994645
Epoch 594: 2021-09-20 22:21:02.176079
Loss: 0.9738233306047742
Train accuracy: 0.2783450246532263
Epoch 595: 2021-09-20 22:23:48.925037
Loss: 0.9836429182599838
Train accuracy: 0.28061866130693747
Epoch 596: 2021-09-20 22:26:35.719750
Loss: 0.9563320219819718
Train accuracy: 0.2726492090620248
Epoch 597: 2021-09

Train accuracy: 0.2802600056515556
Epoch 671: 2021-09-21 01:56:06.478792
Loss: 0.9566249909553705
Train accuracy: 0.28214171856203357
Epoch 672: 2021-09-21 01:58:53.180742
Loss: 0.9958435350138517
Train accuracy: 0.28237239403011616
Epoch 673: 2021-09-21 02:01:40.407675
Loss: 0.9314553544278483
Train accuracy: 0.2804905700706544
Epoch 674: 2021-09-21 02:04:27.153412
Loss: 0.9472661653419644
Train accuracy: 0.28483493161580137
Epoch 675: 2021-09-21 02:07:14.471722
Loss: 0.947806239119903
Train accuracy: 0.28546422139386834
Epoch 676: 2021-09-21 02:10:02.057243
Loss: 0.9456577395491833
Train accuracy: 0.2714111274202401
Epoch 677: 2021-09-21 02:12:49.575429
Loss: 0.9839606356618508
Train accuracy: 0.28211834333210223
Epoch 678: 2021-09-21 02:15:37.076291
Loss: 0.9615128912785023
Train accuracy: 0.28224226507155004
Epoch 679: 2021-09-21 02:18:24.204695
Loss: 0.959589024937005
Train accuracy: 0.2785730585453783
Epoch 680: 2021-09-21 02:21:12.185027
Loss: 0.9607956960339982
Train accuracy: 

Loss: 0.9781458902604935
Train accuracy: 0.2851467962954347
Epoch 755: 2021-09-21 05:50:14.132595
Loss: 0.9587552452729391
Train accuracy: 0.28673755113760147
Epoch 756: 2021-09-21 05:53:00.986203
Loss: 0.9461085566821689
Train accuracy: 0.2868078982339057
Epoch 757: 2021-09-21 05:55:47.623857
Loss: 0.9361073627477576
Train accuracy: 0.285396909570581
Epoch 758: 2021-09-21 05:58:33.727580
Loss: 0.9781735210618806
Train accuracy: 0.2832936332622045
Epoch 759: 2021-09-21 06:01:20.270088
Loss: 0.9503582837751303
Train accuracy: 0.28385522703226657
Epoch 760: 2021-09-21 06:04:07.465355
Loss: 0.9444285877603522
Train accuracy: 0.280161120999721
Epoch 761: 2021-09-21 06:06:55.119305
Loss: 0.9492718586214424
Train accuracy: 0.2766948532825858
Epoch 762: 2021-09-21 06:09:41.511815
Loss: 0.9622170805299471
Train accuracy: 0.28540847290083676
Epoch 763: 2021-09-21 06:12:29.612678
Loss: 0.9333874219862044
Train accuracy: 0.2835715759169964
Epoch 764: 2021-09-21 06:15:16.571730
Loss: 0.96447293538

Epoch 838: 2021-09-21 09:40:41.981271
Loss: 0.9666213488111333
Train accuracy: 0.28402019499782455
Epoch 839: 2021-09-21 09:43:28.289544
Loss: 0.9412263029199682
Train accuracy: 0.2825986242304284
Epoch 840: 2021-09-21 09:46:14.296976
Loss: 0.9484929506255741
Train accuracy: 0.28905247778920007
Epoch 841: 2021-09-21 09:48:59.726973
Loss: 0.9576601878017482
Train accuracy: 0.286480545875417
Epoch 842: 2021-09-21 09:51:45.896971
Loss: 0.9486185362096423
Train accuracy: 0.28773008951076456
Epoch 843: 2021-09-21 09:54:32.356446
Loss: 0.961014131190674
Train accuracy: 0.2880877229265353
Epoch 844: 2021-09-21 09:57:17.954126
Loss: 0.9440609965405201
Train accuracy: 0.2835148389183285
Epoch 845: 2021-09-21 10:00:04.087937
Loss: 0.9383172439721978
Train accuracy: 0.291509650601657
Epoch 846: 2021-09-21 10:02:50.177433
Loss: 0.9351309154394907
Train accuracy: 0.2861626969103684
Epoch 847: 2021-09-21 10:05:36.313548
Loss: 0.9403924069873428
Train accuracy: 0.28721285601587837
Epoch 848: 2021-09-

Train accuracy: 0.28432367889217564
Epoch 922: 2021-09-21 13:33:18.956189
Loss: 0.9341755008156281
Train accuracy: 0.2918133421751693
Epoch 923: 2021-09-21 13:36:04.530023
Loss: 0.9404149579097223
Train accuracy: 0.29033776445563775
Epoch 924: 2021-09-21 13:38:50.217147
Loss: 0.9419279724645634
Train accuracy: 0.2887382123863558
Epoch 925: 2021-09-21 13:41:36.121395
Loss: 0.9366986867250314
Train accuracy: 0.28808699414163713
Epoch 926: 2021-09-21 13:44:22.129175
Loss: 0.9348807784513605
Train accuracy: 0.2834822635204904
Epoch 927: 2021-09-21 13:47:08.146495
Loss: 0.9639438831388394
Train accuracy: 0.2868664619063878
Epoch 928: 2021-09-21 13:49:53.480510
Loss: 0.9373544443714846
Train accuracy: 0.2928249102157259
Epoch 929: 2021-09-21 13:52:39.330331
Loss: 0.9314761570142386
Train accuracy: 0.2841179285448776
Epoch 930: 2021-09-21 13:55:25.901118
Loss: 0.9460228448522101
Train accuracy: 0.28845179714711244
Epoch 931: 2021-09-21 13:58:12.663145
Loss: 0.9184612346177392
Train accuracy: 